In [284]:
from skimage import io, transform
import glob
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
from tensorflow.python import keras
from keras import Sequential
from keras.layers import InputLayer, Input, Reshape, MaxPooling2D, Conv2D, Dense, Flatten

In [285]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE)
NUM_CLASS = 5
NUM_CHANNELS = 1
IMG_SHAPE_FULL = (IMG_SIZE, IMG_SIZE, NUM_CHANNELS)

In [286]:
def read_imgs(path):
    euro_dirs = [path + '/' + x for x in os.listdir(path) if os.path.isdir(path + '/' + x)]
    imgs = list()
    labels = list()
    for idx, folder in enumerate(euro_dirs):
        for filename in glob.glob(folder + '/*.jp*g'):
            img = io.imread(filename, as_gray=True)
            img = transform.resize(img, IMG_SHAPE)
            
            imgs.append(img)
            labels.append(idx)
            
    return np.asarray(imgs, np.float32), np.asarray(labels, np.int32)

imgs, cls = read_imgs('./data/coins')

/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [287]:
labels = np.zeros((cls.size, 5))
labels[np.arange(cls.size), cls] = 1

In [288]:
# Start construction of the Keras Sequential model.
model = Sequential()

# Add an input layer which is similar to a feed_dict in TensorFlow.
# Note that the input-shape must be a tuple containing the image-size.
model.add(InputLayer(input_shape=IMG_SHAPE))

# The input is a flattened array with 784 elements,
# but the convolutional layers expect images with shape (28, 28, 1)
model.add(Reshape(IMG_SHAPE_FULL))

# First convolutional layer with ReLU-activation and max-pooling.
model.add(Conv2D(kernel_size=5, strides=1, filters=16, padding='same',
                 activation='relu', name='layer_conv1'))
model.add(MaxPooling2D(pool_size=2, strides=2))

# Second convolutional layer with ReLU-activation and max-pooling.
model.add(Conv2D(kernel_size=5, strides=1, filters=36, padding='same',
                 activation='relu', name='layer_conv2'))
model.add(MaxPooling2D(pool_size=2, strides=2))

# Flatten the 4-rank output of the convolutional layers
# to 2-rank that can be input to a fully-connected / dense layer.
model.add(Flatten())

# First fully-connected / dense layer with ReLU-activation.
model.add(Dense(128, activation='relu'))

# Last fully-connected / dense layer with softmax-activation
# for use in classification.
model.add(Dense(num_classes, activation='softmax'))

In [289]:
model.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

model.fit(x=imgs,
           y=labels,
           epochs=1, batch_size=128)

Epoch 1/1
651/651 [==============================] - 27s 42ms/step - loss: 10.1237 - acc: 0.2074


In [290]:
result = model.evaluate(x=imgs,
                        y=labels)

651/651 [==============================] - 10s 16ms/step


In [291]:
for name, value in zip(model.metrics_names, result):
    print(name, value)

loss 12.6518383466
acc 0.215053763441
